In [1]:
import shutil
import requests
import sys
from langchain_core.language_models import BaseChatModel
from langchain.docstore.document import Document as LangchainDocument
from langchain_community.document_loaders import PyPDFLoader
import os
import gradio as gr

In [5]:
# HF_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")
HF_TOKEN = ""

In [6]:
from huggingface_hub import login
login(HF_TOKEN)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [31]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "kaist-ai/prometheus-13b-v1.0"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [29]:
from langchain_community.llms import HuggingFaceHub
from langchain_community.chat_models import ChatHuggingFace

llm = HuggingFaceHub(
    repo_id= "mistralai/Mixtral-8x7B-Instruct-v0.1",
    task="text-generation",
    huggingfacehub_api_token = HF_TOKEN,
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

chat_model_generation = ChatHuggingFace(llm=llm)


llm = HuggingFaceHub(
    repo_id= "kaist-ai/prometheus-13b-v1.0",
    task="text-generation",
    huggingfacehub_api_token = HF_TOKEN,
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)


chat_model_evaluation = ChatHuggingFace(llm=llm)

WARNING! repo_id is not default parameter.
                    repo_id was transferred to model_kwargs.
                    Please confirm that repo_id is what you intended.
WARNING! task is not default parameter.
                    task was transferred to model_kwargs.
                    Please confirm that task is what you intended.
WARNING! huggingfacehub_api_token is not default parameter.
                    huggingfacehub_api_token was transferred to model_kwargs.
                    Please confirm that huggingfacehub_api_token is what you intended.
WARNING! repo_id is not default parameter.
                    repo_id was transferred to model_kwargs.
                    Please confirm that repo_id is what you intended.
WARNING! task is not default parameter.
                    task was transferred to model_kwargs.
                    Please confirm that task is what you intended.
WARNING! huggingfacehub_api_token is not default parameter.
                    huggingfacehub_ap

tokenizer_config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

In [22]:
from langchain.prompts import ChatPromptTemplate

resume_summary_generation_prompt = """
Your task is to summarize in 1500 characters the experiences and skills from a given resume in a format that 
facilitates easy comparison with a job description.

Provide your summary as follows:

Output:::
Summary: (your summary of the experience from the resume, structured for easy comparison with a job description)

Now here is the resume.

Resume: {resume}\n
Output:::"""

resume_summary_generation_prompt = ChatPromptTemplate.from_template(resume_summary_generation_prompt)
resume_summary_generation_agent = resume_summary_generation_prompt | chat_model




jobDescription_summary_generation_prompt = """
Your task is to summarize in 1000 characters the provided job description in a concise format that 
can be easily compared with the summary of a resume.

Provide your job description summary as follows:

Output:::
Job Description Summary: (your summarized version of the job description, structured for easy comparison with the summary of a resume)

Now here is job description.

job description: {jobDescription}\n
Output:::"""

jobDescription_summary_generation_prompt = ChatPromptTemplate.from_template(jobDescription_summary_generation_prompt)
jobDescription_summary_generation_agent = jobDescription_summary_generation_prompt | chat_model







In [23]:
def loadTextFromResume(file):
    loader = PyPDFLoader(file)
    pages = loader.load_and_split()
    resume = ""
    i = 0
    for page in pages:
        if i != 0:
            resume += "\n"
        resume += page.page_content
        i += 1
    return resume

In [25]:
def SummarizeResume(resume):
    output = resume_summary_generation_agent.invoke({"resume": resume}).content
    try:
        summary = output.split("Summary:")[2].split("Answer: ")[0]
        return summary
    except Exception as e:
        return "No summary of the resume...."

In [ ]:
def SummarizeJobDescription(jobDescription):
    output = jobDescription_summary_generation_agent.invoke({"jobDescription":jobDescription}).content
    try:
        summary = output.split("job description:")[2].split("Answer: ")[0]
        return summary
    except Exception as e:
        return "No summary of the resume...."

In [ ]:
def evaluate(resumeSummary, jobDescriptionSummary):
    output = QA_generation_agent.invoke({"resume": resume}).content
    try:
        summary = output.split("Summary:")[2].split("Answer: ")[0]
        return summary
    except Exception as e:
        return "No summary of the resume...."

In [26]:
def predict(file, jobDescription):
    
    #Loads Text from resume
    resume = loadTextFromResume(file)

    #summarizes resume text for easy Job description comparison
    resumeSummary = SummarizeResume(resume) 

    #Summarize Job description
    jobDescriptionSummary = SummarizeJobDescription(jobDescription)

    score = evaluate(resumeSummary, jobDescriptionSummary)

    return f"Resume match score is: {score} out of 5"



    
interface = gr.Interface(
    fn=predict,
    inputs=[
        gr.File(label="Upload Resume pdf only"),
        gr.Textbox(label="Job Description"),
    ],
    outputs="text",
    title="JobFitScore"
)

interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7865
Running on public URL: https://2e28c7ee11b81739a9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
